 Virtual Environment with the libraries required.
 - python=3.8
  - scikit-learn
  - jupyter
  - numpy
  - matplotlib
  - seaborn
  - snowflake-snowpark-python
  - pandas

In [23]:
import os
os.getcwd()

'C:\\Users\\Jayanth\\Python_DataAnalysis\\Snowflake'

In [1]:
from snowflake.snowpark import Session
import numpy as np
import pandas as pd

C:\Users\Jayanth\anaconda3\envs\py38_env\lib\site-packages\snowflake\connector\options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (6.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [7]:
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *

import pandas as pd

#from sklearn import linear_model

#import matplotlib.pyplot as plt

#%matplotlib inline
import datetime as dt
import numpy as np
#import seaborn as sns

#Snowflake connection info is saved in config.py
from configbkp1 import snowflake_conn_prop


# lets import some tranformations functions
from snowflake.snowpark.functions import udf, col, lit, translate, is_null, iff
from snowflake.snowpark.functions import month,year,col,sum

In [1]:
### Before running the below statements on the dataset make sure the sql file (02-Lesson.sql)
### has been executed.

In [8]:
from snowflake.snowpark import version
print(version.VERSION)
#session.close()
session = Session.builder.configs(snowflake_conn_prop).create()
session.sql("use role accountadmin").collect()
session.sql("create database if not exists  {}".format(snowflake_conn_prop['database'])).collect()
session.sql("use database {}".format(snowflake_conn_prop['database'])).collect()
session.sql("create schema if not exists {}".format(snowflake_conn_prop['schema'])).collect()
session.sql("use schema {}".format(snowflake_conn_prop['schema'])).collect()
session.sql("create or replace warehouse {} with \
                WAREHOUSE_SIZE = XSMALL \
                AUTO_SUSPEND = 120 \
                AUTO_RESUME = TRUE".format(snowflake_conn_prop['warehouse'])).collect()
session.sql("use warehouse {}".format(snowflake_conn_prop['warehouse']))
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

(1, 2, 0)
[Row(CURRENT_WAREHOUSE()='COMPUTE_WH_1', CURRENT_DATABASE()='DEMO_DB', CURRENT_SCHEMA()='TELCO')]


In [9]:
snow_df_spend = session.table('DASH_DB.DASH_SCHEMA.CAMPAIGN_SPEND')
snow_df_spend_per_channel = snow_df_spend.group_by(year('DATE'), month('DATE'),'CHANNEL').agg(sum('TOTAL_COST').as_('TOTAL_COST')).\
    with_column_renamed('"YEAR(DATE)"',"YEAR").with_column_renamed('"MONTH(DATE)"',"MONTH").sort('YEAR','MONTH')

In [10]:
snow_df_spend_per_channel.show()

---------------------------------------------------
|"YEAR"  |"MONTH"  |"CHANNEL"      |"TOTAL_COST"  |
---------------------------------------------------
|2012    |5        |search_engine  |516431        |
|2012    |5        |video          |516729        |
|2012    |5        |email          |517208        |
|2012    |5        |social_media   |517618        |
|2012    |6        |video          |501098        |
|2012    |6        |search_engine  |506497        |
|2012    |6        |social_media   |504679        |
|2012    |6        |email          |501947        |
|2012    |7        |search_engine  |522780        |
|2012    |7        |email          |518405        |
---------------------------------------------------



In [12]:
snow_df_spend_per_month = snow_df_spend_per_channel.pivot('CHANNEL',['search_engine','social_media','video','email']).sum('TOTAL_COST').sort('YEAR','MONTH')
snow_df_spend_per_month = snow_df_spend_per_month.select(
        col("YEAR"),
        col("MONTH"),
        col("'search_engine'").as_("SEARCH_ENGINE"),
        col("'social_media'").as_("SOCIAL_MEDIA"),
        col("'video'").as_("VIDEO"),
        col("'email'").as_("EMAIL")
    )

print("Total Spend Across All Channels")
# See the output of this command in "PY Output" tab below
snow_df_spend_per_month.show()

Total Spend Across All Channels
---------------------------------------------------------------------------
|"YEAR"  |"MONTH"  |"SEARCH_ENGINE"  |"SOCIAL_MEDIA"  |"VIDEO"  |"EMAIL"  |
---------------------------------------------------------------------------
|2012    |5        |516431           |517618          |516729   |517208   |
|2012    |6        |506497           |504679          |501098   |501947   |
|2012    |7        |522780           |521395          |522762   |518405   |
|2012    |8        |519959           |520537          |520685   |521584   |
|2012    |9        |507211           |507404          |511364   |507363   |
|2012    |10       |518942           |520863          |522768   |519950   |
|2012    |11       |505715           |505221          |505292   |503748   |
|2012    |12       |520148           |520711          |521427   |520724   |
|2013    |1        |522151           |518635          |520583   |521167   |
|2013    |2        |467736           |474679          |4

In [13]:
### Save Transformed Data into Snowflake Table
# Let's save the transformed data into a Snowflake table *SPEND_PER_MONTH*.

snow_df_spend_per_month.write.mode('overwrite').save_as_table('DASH_DB.DASH_SCHEMA.SPEND_PER_MONTH')